In [1]:
import sys,os
project_root = os.path.dirname(os.getcwd())
sys.path.insert(0,project_root)

from src.data_handling.load_data import *
from src.data_handling.preprocessing import *
from src.models.model_setup import *
from src.trainer.training import *
from src.trainer.file_utils import *
from transformers import set_seed

c:\Users\Hector Auvinen\Documents\GitHub\adapter_experiments\adapter_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed_value = 42
set_seed(seed_value)
# supports tasks:
# sst2
# cb
# rte
# sick
# mrpc
# boolq
task = "boolq"
model_name = "bert-base-uncased"
# num_labels = 2
output_dir = "C:/Users/Hector Auvinen/Desktop/eval_results"
#logger = logging.getLogger(__name__)
#logger.setLevel(logging.INFO)

In [3]:
data = load_hf_dataset(task,debug=False)

tokenizer = get_tokenizer(model_name)

encode = get_encoding(task)

Generating test split: 100%|██████████| 3245/3245 [00:00<00:00, 270850.66 examples/s]


using bert tokenizer
getting encoding:
<function encode_boolq at 0x0000028C93DE85E0>


In [4]:
#def preprocess_dataset(dataset,encoding_func,tokenizer):
dataset = preprocess_dataset(data,encode,tokenizer)

Map:   0%|          | 0/9427 [00:00<?, ? examples/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returne

mapped
{'train': ['question', 'passage', 'idx', 'label', 'input_ids', 'token_type_ids', 'attention_mask'], 'validation': ['question', 'passage', 'idx', 'label', 'input_ids', 'token_type_ids', 'attention_mask'], 'test': ['question', 'passage', 'idx', 'label', 'input_ids', 'token_type_ids', 'attention_mask']}


In [5]:
#id2label = {id: label for (id,label) in enumerate(dataset["train"].features["labels"].names)}
#num_labels = len(id2label)
num_labels = get_label_count(dataset)
print("labels",num_labels)
model = setup_model(model_name,num_labels,dataset)

adapter_config = adapters.BnConfig(
                          output_adapter=True,
                          mh_adapter=False,
                          reduction_factor=2,
                          non_linearity="relu")

model = add_clf_adapter(task_name=task,model=model,num_labels=num_labels,adapter_config=adapter_config)

default_args = TrainingParameters()
default_args.lr_scheduler_type = "linear"


labels 2


Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
train_args = get_training_arguments(default_args)

trainer = get_trainer(train_args,dataset,model,early_stopping=3)

trainer.train()

                                                    
  0%|          | 50/35370 [00:58<2:07:54,  4.60it/s]

{'eval_loss': 0.6750044226646423, 'eval_accuracy': 0.5859327217125382, 'eval_runtime': 43.8245, 'eval_samples_per_second': 74.616, 'eval_steps_per_second': 9.333, 'epoch': 0.04}


  0%|          | 100/35370 [01:58<2:17:13,  4.28it/s]

{'eval_loss': 0.6559913754463196, 'eval_accuracy': 0.6220183486238532, 'eval_runtime': 47.9209, 'eval_samples_per_second': 68.237, 'eval_steps_per_second': 8.535, 'epoch': 0.08}


                                                       
  0%|          | 150/35370 [02:56<2:16:12,  4.31it/s]

...

                                                       
  2%|▏         | 750/35370 [14:57<2:09:25,  4.46it/s]

{'eval_loss': 0.6333985924720764, 'eval_accuracy': 0.6629969418960244, 'eval_runtime': 48.1088, 'eval_samples_per_second': 67.971, 'eval_steps_per_second': 8.502, 'epoch': 0.64}


  2%|▏         | 750/35370 [14:57<11:30:16,  1.20s/it]

{'train_runtime': 897.2385, 'train_samples_per_second': 315.2, 'train_steps_per_second': 39.421, 'train_loss': 0.6574611511230469, 'epoch': 0.64}


TrainOutput(global_step=750, training_loss=0.6574611511230469, metrics={'train_runtime': 897.2385, 'train_samples_per_second': 315.2, 'train_steps_per_second': 39.421, 'train_loss': 0.6574611511230469, 'epoch': 0.64})

In [7]:
eval_results = trainer.evaluate()

100%|██████████| 409/409 [00:46<00:00,  8.88it/s]


In [8]:
write_eval_results(eval_results,output_dir,task,trainer,adapter_config)

Writing eval results
{'eval_loss': 0.6164497137069702, 'eval_accuracy': 0.6672782874617736, 'eval_runtime': 46.1558, 'eval_samples_per_second': 70.847, 'eval_steps_per_second': 8.861, 'epoch': 0.64}


In [9]:
"""for attr_name, attr_value in vars(default_args).items():
    print(f"{attr_name}: {attr_value}")"""
"""
    train_args = {"label_names":["labels"],
    "evaluation_strategy":"steps",
    "learning_rate":1e-4,
    "num_train_epochs":1,
    "per_device_train_batch_size":8,
    "per_device_eval_batch_size":8,
    "eval_steps":50,
    "logging_steps":200,
    "output_dir":"/eval_results",
    "overwrite_output_dir":True,
    "remove_unused_columns":False,
    "lr_scheduler_type":'linear',
    "load_best_model_at_end":True,
    "metric_for_best_model" : "accuracy",
    "early_stopping_patience":3,
    "save_total_limit":5
}
"""

'\n    train_args = {"label_names":["labels"],\n    "evaluation_strategy":"steps",\n    "learning_rate":1e-4,\n    "num_train_epochs":1,\n    "per_device_train_batch_size":8,\n    "per_device_eval_batch_size":8,\n    "eval_steps":50,\n    "logging_steps":200,\n    "output_dir":"/eval_results",\n    "overwrite_output_dir":True,\n    "remove_unused_columns":False,\n    "lr_scheduler_type":\'linear\',\n    "load_best_model_at_end":True,\n    "metric_for_best_model" : "accuracy",\n    "early_stopping_patience":3,\n    "save_total_limit":5\n}\n'